# 1. Introduction/Business Problem

### 1-1. Background
 
When you driving to another city, you may come across a terrible traffic jam on the high way, it may be an car accident due to bad weather such as rainy and windy. 
Now, wouldn't it be great if there is something in place that could warn you, given the weather and the road conditions about the possibility of you getting into a 
car accident and how severe it would be, so that you would drive more carefully or even change your travel if you are able to. 

### 1-2. Business Problem
The business problem is to predict severity of an accident using relative varables such as weather and road condition, based on prediction score we can set up the warning system.  
If we could warn the severity level before driving the area to the people, people can paln to reduce or avoid to drive any high severity area.  

<i>End of Introduction and Business Problems

# 2. Data Understanding / Preparation

### 2-1. Data Source  

This data is provided in CSV format. 
Data is came from SDOT Traffic Management Division, Traffic Records Group. 
Provided data is collision data from all years by traffic records.
This includes all types of collisions. \
Collisions will display at the intersection or mid-block of a segment. Timeframe: 2004 to Present.


### 2-2. Data summary

Key attribute information  

|Attribute|Data type,length|Description|
|:------|:---|:---| 
|SEVERITYCODE    |    int64  | A code that corresponds to the severity of the collision:|
|                |           |    • 3—fatality|
|                |           |    • 2b—serious injury|
|                |           |    • 2—injury|
|                |           |    • 1—prop damage|
|                |           |    • 0—unknown|
|OBJECTID           | int64  | ESRI unique identifier |
|INCKEY             | int64  | A unique key for the incident |
|COLDETKEY          | int64  | Secondary key for the incident| 
|ADDRTYPE           | object | Collision address type:|
|                   |        |     • Alley|
|                   |        |     • Block|
|                   |        |     • Intersection|
|INTKEY             |float64 | A unique key for the incident |
|LOCATION           |object  | Description of the general location of the collision |
|SEVERITYDESC       |object  | A detailed description of the severity of the collision|
|PERSONCOUNT        |int64   | The total number of people involved in the collision |
|VEHCOUNT           |Double  | The number of vehicles involved in the collision, This is entered by the state. |


\
\



Data contains 38 columns and 194673 rows.\
check missing data and any adjusment. cleansing data catains 187525 rows. \
I will extract data and review its relationship and impact level of severity. \
Main extracted data with key columns such as SEVERITYCODE	ADDRTYPE	ROADCOND	LIGHTCOND	WEATHER supposed to impact severitycode.\
additionally PERSONCOUNT, VEHCOUNT will be added to dataset for split and test.\
\
review data distribution between severitycode and each attibutes. \
Addrtype shows different trend on severitycode, it may impact the severity. \
ROADCOND shows two attribute have impactful on severity.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
# CSV file tranfer 
!wget -o -- https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

In [21]:
# data review 
data = pd.read_csv('Data-Collisions.csv')
data.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [23]:
data.shape 
# Data contains 38 columns and 194673 rows.

(194673, 38)

In [24]:
data.columns

Index(['SEVERITYCODE', 'X', 'Y', 'OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO',
       'STATUS', 'ADDRTYPE', 'INTKEY', 'LOCATION', 'EXCEPTRSNCODE',
       'EXCEPTRSNDESC', 'SEVERITYCODE.1', 'SEVERITYDESC', 'COLLISIONTYPE',
       'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT', 'VEHCOUNT', 'INCDATE',
       'INCDTTM', 'JUNCTIONTYPE', 'SDOT_COLCODE', 'SDOT_COLDESC',
       'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
       'PEDROWNOTGRNT', 'SDOTCOLNUM', 'SPEEDING', 'ST_COLCODE', 'ST_COLDESC',
       'SEGLANEKEY', 'CROSSWALKKEY', 'HITPARKEDCAR'],
      dtype='object')

In [121]:
data.describe()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,INTKEY,SEVERITYCODE.1,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,SDOT_COLCODE,SDOTCOLNUM,SEGLANEKEY,CROSSWALKKEY
count,194673.000000,189339.000000,189339.000000,194673.000000,194673.000000,194673.000000,65070.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,194673.000000,1.149360e+05,194673.000000,1.946730e+05
mean,1.298901,-122.330518,47.619543,108479.364930,141091.456350,141298.811381,37558.450576,1.298901,2.444427,0.037139,0.028391,1.920780,13.867768,7.972521e+06,269.401114,9.782452e+03
std,0.457778,0.029976,0.056157,62649.722558,86634.402737,86986.542110,51745.990273,0.457778,1.345929,0.198150,0.167413,0.631047,6.868755,2.553533e+06,3315.776055,7.226926e+04
min,1.000000,-122.419091,47.495573,1.000000,1001.000000,1001.000000,23807.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.007024e+06,0.000000,0.000000e+00
25%,1.000000,-122.348673,47.575956,54267.000000,70383.000000,70383.000000,28667.000000,1.000000,2.000000,0.000000,0.000000,2.000000,11.000000,6.040015e+06,0.000000,0.000000e+00
50%,1.000000,-122.330224,47.615369,106912.000000,123363.000000,123363.000000,29973.000000,1.000000,2.000000,0.000000,0.000000,2.000000,13.000000,8.023022e+06,0.000000,0.000000e+00
75%,2.000000,-122.311937,47.663664,162272.000000,203319.000000,203459.000000,33973.000000,2.000000,3.000000,0.000000,0.000000,2.000000,14.000000,1.015501e+07,0.000000,0.000000e+00
max,2.000000,-122.238949,47.734142,219547.000000,331454.000000,332954.000000,757580.000000,2.000000,81.000000,6.000000,2.000000,12.000000,69.000000,1.307202e+07,525241.000000,5.239700e+06


In [123]:
data.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [145]:
# extract relative data
df=data.loc[:,['SEVERITYCODE','ADDRTYPE','ROADCOND','LIGHTCOND','WEATHER','PERSONCOUNT']]
df.head()

,SEVERITYCODE,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER,PERSONCOUNT
0,2,Intersection,Wet,Daylight,Overcast,2
1,1,Block,Wet,Dark - Street Lights On,Raining,2
2,1,Block,Dry,Daylight,Overcast,4
3,1,Block,Dry,Daylight,Clear,3
4,2,Intersection,Wet,Daylight,Raining,2


In [ ]:
# Cleaning missing data 
# group by tables 
# check corelation of each items with severity code

In [146]:
df.dropna(axis=0, inplace=True)
df.head()

,SEVERITYCODE,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER,PERSONCOUNT
0,2,Intersection,Wet,Daylight,Overcast,2
1,1,Block,Wet,Dark - Street Lights On,Raining,2
2,1,Block,Dry,Daylight,Overcast,4
3,1,Block,Dry,Daylight,Clear,3
4,2,Intersection,Wet,Daylight,Raining,2


In [30]:
df.shape

(187525, 5)

In [144]:
# check the total frame of extracted data
# df.groupby(['ADDRTYPE','ROADCOND','LIGHTCOND','WEATHER'])['SEVERITYCODE'].value_counts().to_frame()

In [147]:
# check the severity code group and the counts. 
df_sev = df.groupby(['SEVERITYCODE'], as_index=True).count()
df_sev

,ADDRTYPE,ROADCOND,LIGHTCOND,WEATHER,PERSONCOUNT
SEVERITYCODE,,,,,
1,130642,130642,130642,130642,130642
2,56883,56883,56883,56883,56883


In [139]:
# check the severity code with ADDRTYPE
Add=df.groupby(['SEVERITYCODE'])['ADDRTYPE'].value_counts().to_frame()
Add

ADDRTYPE
SEVERITYCODE ADDRTYPE              
1            Block            93892
             Intersection     36088
             Alley              662
2            Block            29429
             Intersection     27374
             Alley               80

In [140]:
# different type of table 
Add=df['SEVERITYCODE'].groupby([df['SEVERITYCODE'], df['ADDRTYPE']]).count()
Add.unstack()

ADDRTYPE,Alley,Block,Intersection
SEVERITYCODE,,,
1,662,93892,36088
2,80,29429,27374


In [141]:
# check the severity code with ROADCOND
Road=df['SEVERITYCODE'].groupby([df['SEVERITYCODE'], df['ROADCOND']]).count()
Road.unstack()

ROADCOND,Dry,Ice,Oil,Other,Sand/Mud/Dirt,Snow/Slush,Standing Water,Unknown,Wet
SEVERITYCODE,,,,,,,,,
1,83835,923,40,82,51,827,82,13279,31523
2,39901,270,24,42,22,165,29,730,15700


In [142]:
# check the severity code with LIGHTCOND
Light=df['SEVERITYCODE'].groupby([df['SEVERITYCODE'], df['LIGHTCOND']]).count()
Light.unstack()

LIGHTCOND,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Unknown
SEVERITYCODE,,,,,,,,,
1,1192,869,33816,7,1668,76998,3907,175,12010
2,334,315,14420,4,823,38410,1936,52,589


In [143]:
# check the severity code with WEATHER
Wea=df['SEVERITYCODE'].groupby([df['SEVERITYCODE'], df['WEATHER']]).count()
Wea.unstack()

WEATHER,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,Raining,Severe Crosswind,Sleet/Hail/Freezing Rain,Snowing,Unknown
SEVERITYCODE,,,,,,,,,,,
1,36,74778,377,676,18836,2,21836,18,85,729,13269
2,13,35721,186,114,8715,3,11140,7,27,167,790


<i> End of Data Section